### Not predicting correctly

Not a correct way as includes the laptimes, want to predict pre race

In [ ]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.multioutput import MultiOutputClassifier
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "../")))
from DB.models import init_db, Circuit, Season, RacingWeekend, Driver, Session, SessionResult, Lap

In [38]:
def create_dataframe():
    # initialize db connection and session
    db_engine, db_session = init_db()

    # query data from the database
    query = db_session.query(
        RacingWeekend.year,
        RacingWeekend.round,
        Circuit.circuit_name,
        Driver.driver_name,
        Driver.driver_short,
        Lap.lap_num,
        Lap.lap_time,
        Lap.tyre,
        Lap.pit,
        Session.session_type
    ).join(RacingWeekend.circuit) \
     .join(RacingWeekend.sessions) \
     .join(Session.laps) \
     .join(Lap.driver) \
     .join(RacingWeekend.season) \
     .all()

    # convert result to list of dicts
    data = []
    for row in query:
        data.append({
            'year': row.year,
            'round': row.round,
            'circuit_name': row.circuit_name,
            'driver_name': row.driver_name,
            'driver_short': row.driver_short,
            'lap_num': row.lap_num,
            'lap_time': row.lap_time,
            'tyre': row.tyre,
            'pit': row.pit,
            'session_type': row.session_type
        })

    # create dataframe
    df = pd.DataFrame(data)

    # encode 'tyre' and 'pit' as categories
    df['tyre'] = df['tyre'].astype('category')

    # one-hot encode categorical variables
    df = pd.get_dummies(df, columns=['circuit_name', 'driver_name', 'driver_short', 'session_type'], drop_first=True)

    return df

# create the dataframe and print it
df = create_dataframe()


In [39]:
# split data into training (2019, 2020, 2021) and testing (2022)
train_data = df[df['year'].isin([2019, 2020, 2021])]
test_data = df[df['year'] == 2022]

# define features (X) and targets (y) for training
X_train = train_data.drop(columns=['pit', 'tyre'])
y_train = train_data[['pit', 'tyre']]

# define features (X) and targets (y) for testing
X_test = test_data.drop(columns=['pit', 'tyre'])
y_test = test_data[['pit', 'tyre']]

# train the multioutput random forest model
rf_classifier = RandomForestClassifier(n_estimators=100, random_state=42)
multi_output_rf = MultiOutputClassifier(rf_classifier)
multi_output_rf.fit(X_train, y_train)

# predict on the entire 2022 season
y_pred_test = multi_output_rf.predict(X_test)

# add predictions to the test data
test_data['predicted_pit'] = y_pred_test[:, 0]
test_data['predicted_tyre'] = y_pred_test[:, 1]


/tmp/ipykernel_76573/2284921435.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_data['predicted_pit'] = y_pred_test[:, 0]
/tmp/ipykernel_76573/2284921435.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_data['predicted_tyre'] = y_pred_test[:, 1]


In [40]:
# accuracy for both pit and tyre
accuracy_pit = accuracy_score(y_test['pit'], y_pred_test[:, 0])
accuracy_tyre = accuracy_score(y_test['tyre'], y_pred_test[:, 1])
print(f"Accuracy (Pit): {accuracy_pit:.2f}")
print(f"Accuracy (Tyre): {accuracy_tyre:.2f}")

# confusion matrix for both pit and tyre
conf_matrix_pit = confusion_matrix(y_test['pit'], y_pred_test[:, 0])
conf_matrix_tyre = confusion_matrix(y_test['tyre'], y_pred_test[:, 1])

print("\nConfusion Matrix (Pit):")
print(conf_matrix_pit)

print("\nConfusion Matrix (Tyre):")
print(conf_matrix_tyre)

# classification report for both pit and tyre
class_report_pit = classification_report(y_test['pit'], y_pred_test[:, 0])
class_report_tyre = classification_report(y_test['tyre'], y_pred_test[:, 1])

print("\nClassification Report (Pit):")
print(class_report_pit)

print("\nClassification Report (Tyre):")
print(class_report_tyre)


Accuracy (Pit): 0.94
Accuracy (Tyre): 0.51

Confusion Matrix (Pit):
[[46509   970]
 [ 1833   670]]

Confusion Matrix (Tyre):
[[    0   280   464   132     0     0]
 [   86 11694  3550  2244   165    14]
 [  175  4485  7047  4608    86     1]
 [    2  1638  3038  6441    26     0]
 [    0  1450   644   597   305    17]
 [    0   506   194    93     0     0]]

Classification Report (Pit):
              precision    recall  f1-score   support

       False       0.96      0.98      0.97     47479
        True       0.41      0.27      0.32      2503

    accuracy                           0.94     49982
   macro avg       0.69      0.62      0.65     49982
weighted avg       0.93      0.94      0.94     49982


Classification Report (Tyre):
              precision    recall  f1-score   support

          -1       0.00      0.00      0.00       876
           1       0.58      0.66      0.62     17753
           2       0.47      0.43      0.45     16402
           3       0.46      0.58  

In [41]:
# specify driver and race to analyze
driver = 'VER'
race_year = 2022
race_round = 6

# filter test data for the specific race and driver
df_race_driver = test_data[(
    test_data['session_type_Race'] == 1) &  # race sessions only
    (test_data[f'driver_short_{driver}'] == 1) &  # driver filter (VER)
    (test_data['year'] == race_year) &  # year filter
    (test_data['round'] == race_round)  # round filter
]

# get the laps where the driver actually pitted
actual_pit_laps = df_race_driver[df_race_driver['pit'] == 1][['lap_num', 'pit', 'tyre']]

# get the laps where the model predicted a pit stop
predicted_pit_laps = df_race_driver[df_race_driver['predicted_pit'] == 1][['lap_num', 'predicted_pit', 'predicted_tyre']]

# define tyre mapping
tyre_mapping = {
    1: 'SOFT',
    2: 'MEDIUM',
    3: 'HARD',
    4: 'INTERMEDIATE',
    5: 'WET',
}

# function to decode tyre value from integer
def decode_tyres(value):
    return tyre_mapping.get(value, None)  # return None if no matching tyre value

# apply the decoding function to actual and predicted pit laps
actual_pit_laps['decoded_tyre'] = actual_pit_laps['tyre'].apply(decode_tyres)
predicted_pit_laps['decoded_tyre'] = predicted_pit_laps['predicted_tyre'].apply(decode_tyres)

# print actual and predicted pit stops
print(f"Driver {driver} - Actual Pit Stops in Race {race_year} Round {race_round}:")
print(actual_pit_laps[['lap_num', 'pit', 'decoded_tyre']])

print(f"\nDriver {driver} - Predicted Pit Stops in Race {race_year} Round {race_round}:")
print(predicted_pit_laps[['lap_num', 'predicted_pit', 'decoded_tyre']])


Driver VER - Actual Pit Stops in Race 2022 Round 6:
        lap_num   pit decoded_tyre
154724       14  True       MEDIUM
154739       29  True         SOFT
154755       45  True       MEDIUM

Driver VER - Predicted Pit Stops in Race 2022 Round 6:
        lap_num  predicted_pit decoded_tyre
154739       29            1.0       MEDIUM
154755       45            1.0       MEDIUM
